In [39]:
import nltk
from nltk.tokenize import treebank
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 
import math
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import random
from sklearn.feature_extraction.text import CountVectorizer
import string
import re
ngram_vectorizer = CountVectorizer(binary=True, ngram_range=(1, 2))

In [40]:
stopword = set(stopwords.words('english'))

In [41]:
dataset = pd.read_csv('deceptive-opinion.csv')
def clean_text(text):
    text_lc = "".join([word.lower() for word in text ]) # remove puntuation
    text_rc = re.sub('[0-9]+', '', text_lc)
    tokens = re.split('\W+', text_rc)    # tokenization
    text = [word for word in tokens if word not in stopword ]  # remove stopwords and stemming
    return text
def listtostr(s):
    str1=" "
    return (str1.join(s))
def remove_stop(text):
    text_lc = "".join([word.lower() for word in text ])
    tokens = re.split('\W+', text_lc)
    text=[word for word in tokens if word not in stopword]
    return text
dataset['data_cleaned'] = dataset['text'].apply(lambda x: clean_text(x))
dataset['data_no_stop'] = dataset['text'].apply(lambda x: remove_stop(x))
dataset['data_cleaned_str'] = dataset['data_cleaned'].apply(lambda x: listtostr(x))
dataset['data_str_nostop'] = dataset['data_no_stop'].apply(lambda x: listtostr(x))
dataset

,deceptive,hotel,polarity,source,text,data_cleaned,data_no_stop,data_cleaned_str,data_str_nostop
0,truthful,conrad,positive,TripAdvisor,We stayed for a one night getaway with family ...,"[stayed, one, night, getaway, family, thursday...","[stayed, one, night, getaway, family, thursday...",stayed one night getaway family thursday tripl...,stayed one night getaway family thursday tripl...
1,truthful,hyatt,positive,TripAdvisor,Triple A rate with upgrade to view room was le...,"[triple, rate, upgrade, view, room, less, also...","[triple, rate, upgrade, view, room, less, 200,...",triple rate upgrade view room less also includ...,triple rate upgrade view room less 200 also in...
2,truthful,hyatt,positive,TripAdvisor,This comes a little late as I'm finally catchi...,"[comes, little, late, finally, catching, revie...","[comes, little, late, finally, catching, revie...",comes little late finally catching reviews pas...,comes little late finally catching reviews pas...
3,truthful,omni,positive,TripAdvisor,The Omni Chicago really delivers on all fronts...,"[omni, chicago, really, delivers, fronts, spac...","[omni, chicago, really, delivers, fronts, spac...",omni chicago really delivers fronts spaciousne...,omni chicago really delivers fronts spaciousne...
4,truthful,hyatt,positive,TripAdvisor,I asked for a high floor away from the elevato...,"[asked, high, floor, away, elevator, got, room...","[asked, high, floor, away, elevator, got, room...",asked high floor away elevator got room pleasa...,asked high floor away elevator got room pleasa...
...,...,...,...,...,...,...,...,...,...
1595,deceptive,intercontinental,negative,MTurk,Problems started when I booked the InterContin...,"[problems, started, booked, intercontinental, ...","[problems, started, booked, intercontinental, ...",problems started booked intercontinental chica...,problems started booked intercontinental chica...
1596,deceptive,amalfi,negative,MTurk,The Amalfi Hotel has a beautiful website and i...,"[amalfi, hotel, beautiful, website, interior, ...","[amalfi, hotel, beautiful, website, interior, ...",amalfi hotel beautiful website interior decora...,amalfi hotel beautiful website interior decora...
1597,deceptive,intercontinental,negative,MTurk,The Intercontinental Chicago Magnificent Mile ...,"[intercontinental, chicago, magnificent, mile,...","[intercontinental, chicago, magnificent, mile,...",intercontinental chicago magnificent mile outs...,intercontinental chicago magnificent mile outs...
1598,deceptive,palmer,negative,MTurk,"The Palmer House Hilton, while it looks good i...","[palmer, house, hilton, looks, good, pictures,...","[palmer, house, hilton, looks, good, pictures,...",palmer house hilton looks good pictures outsid...,palmer house hilton looks good pictures outsid...


In [42]:
truthful_prior_log_prob = math.log((list(dataset['deceptive']).count("truthful"))/len(dataset))
deceptive_prior_log_prob = math.log((list(dataset['deceptive']).count("deceptive"))/len(dataset))

In [43]:
def train_test_split(df,test_size):
    if isinstance(test_size,float):
        test_size = round(test_size * len(df))
        
    if(test_size > len(df)):
        return 0, df

    indices = df.index.tolist()
    test_indices = random.sample(population=indices,k = test_size)

    test_df = df.loc[test_indices]
    train_df = df.drop(test_indices)
    
    return train_df, test_df


def bigrams_convert(text):
    word = re.split('\W+', text)
    number_of_words = len(word)
    list_of_bigrams = []

    list_of_bigrams=[word[i] + ' ' + word[i+1] for i in range(number_of_words-1)]
    
    return list_of_bigrams
        
dataset['data_bigrams'] = dataset['data_cleaned_str'].apply(lambda x: bigrams_convert(x))

train_df,test_df = train_test_split(dataset,0.3)
        
new=train_df['data_bigrams']

test_df


,deceptive,hotel,polarity,source,text,data_cleaned,data_no_stop,data_cleaned_str,data_str_nostop,data_bigrams
25,truthful,conrad,positive,TripAdvisor,"Arrived on 06/04/05, greeted by friendly doorm...","[arrived, greeted, friendly, doorman, zipped, ...","[arrived, 06, 04, 05, greeted, friendly, doorm...",arrived greeted friendly doorman zipped th flo...,arrived 06 04 05 greeted friendly doorman zipp...,"[arrived greeted, greeted friendly, friendly d..."
1214,deceptive,conrad,negative,MTurk,I will never stay at Conrad Chicago again. The...,"[never, stay, conrad, chicago, employees, rude...","[never, stay, conrad, chicago, employees, rude...",never stay conrad chicago employees rude begin...,never stay conrad chicago employees rude begin...,"[never stay, stay conrad, conrad chicago, chic..."
1019,truthful,ambassador,negative,Web,I've used Priceline for every trip I've made b...,"[used, priceline, every, trip, made, back, chi...","[used, priceline, every, trip, made, back, chi...",used priceline every trip made back chicago fo...,used priceline every trip made back chicago fo...,"[used priceline, priceline every, every trip, ..."
1483,deceptive,monaco,negative,MTurk,The hotel is located in an hard to fins locati...,"[hotel, located, hard, fins, location, chicago...","[hotel, located, hard, fins, location, chicago...",hotel located hard fins location chicago dinin...,hotel located hard fins location chicago dinin...,"[hotel located, located hard, hard fins, fins ..."
1050,truthful,monaco,negative,Web,Over-hyped and over-priced. The fact that they...,"[hyped, priced, fact, complimentary, wine, rec...","[hyped, priced, fact, complimentary, wine, rec...",hyped priced fact complimentary wine reception...,hyped priced fact complimentary wine reception...,"[hyped priced, priced fact, fact complimentary..."
...,...,...,...,...,...,...,...,...,...,...
440,deceptive,fairmont,positive,MTurk,We recently stayed at the Fairmont Chicago Mil...,"[recently, stayed, fairmont, chicago, millenni...","[recently, stayed, fairmont, chicago, millenni...",recently stayed fairmont chicago millennium pa...,recently stayed fairmont chicago millennium pa...,"[recently stayed, stayed fairmont, fairmont ch..."
229,truthful,ambassador,positive,TripAdvisor,The Ambassador East is located in the Gold Coa...,"[ambassador, east, located, gold, coast, neigh...","[ambassador, east, located, gold, coast, neigh...",ambassador east located gold coast neighborhoo...,ambassador east located gold coast neighborhoo...,"[ambassador east, east located, located gold, ..."
625,deceptive,talbott,positive,MTurk,"excellent staff and customer service, very cle...","[excellent, staff, customer, service, clean, s...","[excellent, staff, customer, service, clean, s...",excellent staff customer service clean spotles...,excellent staff customer service clean spotles...,"[excellent staff, staff customer, customer ser..."
774,deceptive,amalfi,positive,MTurk,I recently stayed at the Amalfi Hotel Chicago ...,"[recently, stayed, amalfi, hotel, chicago, chi...","[recently, stayed, amalfi, hotel, chicago, chi...",recently stayed amalfi hotel chicago chicago i...,recently stayed amalfi hotel chicago chicago i...,"[recently stayed, stayed amalfi, amalfi hotel,..."


In [44]:
vocab = {}
truthful_token_count = {} #how many occurances of each word for truthful class
deceptive_token_count = {}

for i in range(len(train_df)):
    #review = list(train_df['data_cleaned_str'])[i]
   # words = word_tokenize(review)
    new_words =train_df['data_bigrams']
    #new=[]
    #new.append(new_words)
    new_words = [y for x in new_words for y in x]
   
    

    for word in new_words[i]:
        if word not in vocab:
            vocab[word] = 1
        else:
            vocab[word] += 1
        if list(train_df['deceptive'])[i] == "truthful":
            if word not in truthful_token_count:
                truthful_token_count[word] = 1
            else:
                truthful_token_count[word] += 1
            if word not in deceptive_token_count:
                deceptive_token_count[word] = 0
        else:
            if word not in deceptive_token_count:
                deceptive_token_count[word] = 1
            else:
                deceptive_token_count[word] += 1
            if word not in truthful_token_count:
                truthful_token_count[word] = 0
                
new_words

['stayed one',
 'one night',
 'night getaway',
 'getaway family',
 'family thursday',
 'thursday triple',
 'triple aaa',
 'aaa rate',
 'rate steal',
 'steal th',
 'th floor',
 'floor room',
 'room complete',
 'complete plasma',
 'plasma tv',
 'tv bose',
 'bose stereo',
 'stereo voss',
 'voss evian',
 'evian water',
 'water gorgeous',
 'gorgeous bathroom',
 'bathroom tub',
 'tub fine',
 'fine us',
 'us concierge',
 'concierge helpful',
 'helpful cannot',
 'cannot beat',
 'beat location',
 'location flaw',
 'flaw breakfast',
 'breakfast pricey',
 'pricey service',
 'service slow',
 'slow hours',
 'hours four',
 'four kids',
 'kids four',
 'four adults',
 'adults friday',
 'friday morning',
 'morning even',
 'even though',
 'though two',
 'two tables',
 'tables restaurant',
 'restaurant food',
 'food good',
 'good worth',
 'worth wait',
 'wait would',
 'would return',
 'return heartbeat',
 'heartbeat gem',
 'gem chicago',
 'chicago ',
 'comes little',
 'little late',
 'late finally',
 'fi

In [45]:
def laplace_smoothening(word,c):
    if(c):
        numerator = truthful_token_count[word] + 1
        denominator = sum(truthful_token_count.values()) + len(vocab)
        return math.log(numerator/denominator)
    else:
        numerator = deceptive_token_count[word] + 1
        denominator = sum(truthful_token_count.values()) + len(vocab)
        return math.log(numerator/denominator)

In [46]:
def predict(reviews):
    predicted_classes = []
    for review in reviews:
        
        class_scores = [deceptive_prior_log_prob,truthful_prior_log_prob]
        words = word_tokenize(review)
        for word in words:
            if word not in vocab:
                continue
            
            for c in range(2):
                log_w_give_c = laplace_smoothening(word,c)
                class_scores[c] += log_w_give_c
        
        predicted_classes.append(class_scores.index(max(class_scores)))  
    return predicted_classes

In [47]:
def addLabel(value):
    if(value == "truthful"):
        return 1
    else:
        return 0

test_df['label'] = test_df['deceptive'].apply(addLabel)

In [48]:
true_classes = list(test_df.label)

In [49]:
predicted_classes = predict(test_df.text)

In [50]:
count = 0
for i in range(len(true_classes)):
    if true_classes[i] == predicted_classes[i]:
        count += 1
count/len(true_classes)

0.46458333333333335

In [36]:
truePositive = trueNegative = falsePositive = falseNegative = 0
    

In [37]:
for i in range(len(true_classes)):
    if(true_classes[i] == predicted_classes[i] == 0):
        truePositive += 1
    elif true_classes[i] == predicted_classes[i] == 1:
        trueNegative += 1
    elif true_classes[i] == 1 and predicted_classes[i] == 0:
        falsePositive += 1
    elif true_classes[i] == 0 and predicted_classes[i] == 1 :
        falseNegative += 1

In [38]:
accuracy = (truePositive + trueNegative)/(truePositive + trueNegative + falsePositive + falseNegative)
precision = truePositive/(truePositive + falsePositive)
recall =truePositive/(truePositive + falseNegative) 

accuracy,precision,recall

(0.525, 0.5471698113207547, 0.12446351931330472)